# Calculating Gradient Models (Gaussian filter))
$$\tau_{ij} = \overline{u_iu}_j - \overline{u}_i \overline{u}_j$$ 
$$  \sigma_{i} = \overline{u_i\omega} - \overline{u}_i\overline{\omega}$$
$$\Pi_{\omega} = \nabla \cdot \sigma $$

$$\tau_{ij}^{\text{GM}} = \frac{1}{1!}\frac{\Delta^2}{12} \left( \frac{\partial \overline{u}_i}{\partial x_k} \frac{\partial \overline{u}_j}{\partial x_k}  \right) + 
\frac{1}{2!}\frac{\Delta^4}{12^2}
\left(\frac{\partial^2 \overline{u}_i}{\partial x_k x_l}\frac{\partial^2 \overline{u}_j}{\partial x_k \partial x_l} \right) +
\frac{1}{3!}\frac{\Delta^6}{12^3}\left(\frac{\partial^3 \overline{u}_i}{\partial x_k \partial x_l \partial x_m }\frac{\partial^3 \overline{u}_j}{\partial x_k \partial x_l \partial x_m} \right)$$
\
Here, we compute the GM models and validate the code.

In [1]:
# Import necessary libraries and modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from scipy.io import loadmat

# Importing custom modules 
from py2d.filter import filter2D
from py2d.SGSterms import Tau, Sigma, PiOmega, PiUV
from py2d.convert import Omega2Psi, Psi2UV
from py2d.initialize import initialize_wavenumbers_rfft2
from py2d.gradient_model import TauGM2, TauGM4, TauGM6, SigmaGM2, SigmaGM4, SigmaGM6, PiOmegaGM2, PiOmegaGM4, PiOmegaGM6

# Assuming Omega_DNS is provided and is 2D_matrix
# Assuming Kx and Ky are provided

# Define parameters 
nx = 1024
filename = '../data/ICs/NX' + str(nx) + '/1.mat'
Lx = 2 * np.pi
N_LES = 128*np.ones(2, dtype=int)
Delta = 2*Lx/N_LES[0]

dealias = True

filter = 'gaussian'
coarseGrain = 'spectral'

# Load the data from the file
data = loadmat(filename)
# Extract the Omega data from the file
Omega_DNS = data['Omega']


# Initialize wavenumbers
Kx, Ky, Ksq, _, invKsq = initialize_wavenumbers_rfft2(nx, nx, Lx, Lx, INDEXING='ij')
Psi_DNS = Omega2Psi(Omega_DNS, invKsq)
U_DNS, V_DNS = Psi2UV(Psi_DNS, Kx, Ky)

if coarseGrain == 'spectral':
    Kx_coarse, Ky_coarse, Ksq_coarse, _, _ = initialize_wavenumbers_rfft2(N_LES[0], N_LES[1], Lx, Lx, INDEXING='ij')
elif coarseGrain == None:
    Kx_coarse, Ky_coarse, Ksq_coarse, _, _ = initialize_wavenumbers_rfft2(nx, nx, Lx, Lx, INDEXING='ij')

Omegaf = filter2D(Omega_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, Ngrid=N_LES, spectral=False)
Uf = filter2D(U_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, Ngrid=N_LES, spectral=False)
Vf = filter2D(V_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, Ngrid=N_LES, spectral=False)


Tau11, Tau12, Tau22 = Tau(Omega_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, N_LES=N_LES, dealias=dealias)
Tau11GM2, Tau12GM2, Tau22GM2 = TauGM2(Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
Tau11GM4, Tau12GM4, Tau22GM4 = TauGM4(Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
Tau11GM6, Tau12GM6, Tau22GM6 = TauGM6(Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)

Sigma1, Sigma2 = Sigma(Omega_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, N_LES=N_LES, dealias=dealias)
Sigma1GM2, Sigma2GM2 = SigmaGM2(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
Sigma1GM4, Sigma2GM4 = SigmaGM4(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
Sigma1GM6, Sigma2GM6 = SigmaGM6(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)

PiOmega_val = PiOmega(Omega_DNS, filterType=filter, coarseGrainType=coarseGrain, Delta=Delta, N_LES=N_LES, dealias=dealias)
PiOmegaGM2 = PiOmegaGM2(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
PiOmegaGM4 = PiOmegaGM4(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)
PiOmegaGM6 = PiOmegaGM6(Omegaf, Uf, Vf, Kx_coarse, Ky_coarse, Delta, filterType=filter, spectral=False, dealias=dealias)


Validating the GM codes by calculating correlation and MSE between the FDNS (Truth) and GM models
$$\tau_{ij}, \sigma_i, \Pi_{\omega}$$

In [2]:
# Compute the correlation coefficient and the mean squared error

# Tau
corr_Tau11GM2, _ = pearsonr(Tau11.flatten(), Tau11GM2.flatten())
corr_Tau12GM2, _ = pearsonr(Tau12.flatten(), Tau12GM2.flatten())
corr_Tau22GM2, _ = pearsonr(Tau22.flatten(), Tau22GM2.flatten())

corr_Tau11GM4, _ = pearsonr(Tau11.flatten(), Tau11GM4.flatten())
corr_Tau12GM4, _ = pearsonr(Tau12.flatten(), Tau12GM4.flatten())
corr_Tau22GM4, _ = pearsonr(Tau22.flatten(), Tau22GM4.flatten())

corr_Tau11GM6, _ = pearsonr(Tau11.flatten(), Tau11GM6.flatten())
corr_Tau12GM6, _ = pearsonr(Tau12.flatten(), Tau12GM6.flatten())
corr_Tau22GM6, _ = pearsonr(Tau22.flatten(), Tau22GM6.flatten())

MSE_Tau11GM2 = mean_squared_error(Tau11.flatten(), Tau11GM2.flatten())
MSE_Tau12GM2 = mean_squared_error(Tau12.flatten(), Tau12GM2.flatten())
MSE_Tau22GM2 = mean_squared_error(Tau22.flatten(), Tau22GM2.flatten())

MSE_Tau11GM4 = mean_squared_error(Tau11.flatten(), Tau11GM4.flatten())
MSE_Tau12GM4 = mean_squared_error(Tau12.flatten(), Tau12GM4.flatten())
MSE_Tau22GM4 = mean_squared_error(Tau22.flatten(), Tau22GM4.flatten())

MSE_Tau11GM6 = mean_squared_error(Tau11.flatten(), Tau11GM6.flatten())
MSE_Tau12GM6 = mean_squared_error(Tau12.flatten(), Tau12GM6.flatten())
MSE_Tau22GM6 = mean_squared_error(Tau22.flatten(), Tau22GM6.flatten())

# Sigma
corr_Sigma1GM2, _ = pearsonr(Sigma1.flatten(), Sigma1GM2.flatten())
corr_Sigma2GM2, _ = pearsonr(Sigma2.flatten(), Sigma2GM2.flatten())

corr_Sigma1GM4, _ = pearsonr(Sigma1.flatten(), Sigma1GM4.flatten())
corr_Sigma2GM4, _ = pearsonr(Sigma2.flatten(), Sigma2GM4.flatten())

corr_Sigma1GM6, _ = pearsonr(Sigma1.flatten(), Sigma1GM6.flatten())
corr_Sigma2GM6, _ = pearsonr(Sigma2.flatten(), Sigma2GM6.flatten())

MSE_Sigma1GM2 = mean_squared_error(Sigma1.flatten(), Sigma1GM2.flatten())
MSE_Sigma2GM2 = mean_squared_error(Sigma2.flatten(), Sigma2GM2.flatten())

MSE_Sigma1GM4 = mean_squared_error(Sigma1.flatten(), Sigma1GM4.flatten())
MSE_Sigma2GM4 = mean_squared_error(Sigma2.flatten(), Sigma2GM4.flatten())

MSE_Sigma1GM6 = mean_squared_error(Sigma1.flatten(), Sigma1GM6.flatten())
MSE_Sigma2GM6 = mean_squared_error(Sigma2.flatten(), Sigma2GM6.flatten())

# PiOmega

corr_PiOmegaGM2, _ = pearsonr(PiOmega_val.flatten(), PiOmegaGM2.flatten())
corr_PiOmegaGM4, _ = pearsonr(PiOmega_val.flatten(), PiOmegaGM4.flatten())
corr_PiOmegaGM6, _ = pearsonr(PiOmega_val.flatten(), PiOmegaGM6.flatten())

MSE_PiOmegaGM2 = mean_squared_error(PiOmega_val.flatten(), PiOmegaGM2.flatten())
MSE_PiOmegaGM4 = mean_squared_error(PiOmega_val.flatten(), PiOmegaGM4.flatten())
MSE_PiOmegaGM6 = mean_squared_error(PiOmega_val.flatten(), PiOmegaGM6.flatten())

# Print the results
print('Calculate the correlations between the FDNS and GM2/4/6 SGS terms')

print('Correlation coefficient between Tau11 and GM2, GM4, GM6: ', corr_Tau11GM2, corr_Tau11GM4, corr_Tau11GM6)
print('Mean squared error between Tau11 and GM2, GM4, GM6: ', MSE_Tau11GM2, MSE_Tau11GM4, MSE_Tau11GM6)
print('Correlation coefficient between Tau12 and GM2, GM4, GM6: ', corr_Tau12GM2, corr_Tau12GM4, corr_Tau12GM6)
print('Mean squared error between Tau12 and GM2, GM4, GM6: ', MSE_Tau12GM2, MSE_Tau12GM4, MSE_Tau12GM6)
print('Correlation coefficient between Tau22 and GM2, GM4, GM6: ', corr_Tau22GM2, corr_Tau22GM4, corr_Tau22GM6)
print('Mean squared error between Tau22 and GM2, GM4, GM6: ', MSE_Tau22GM2, MSE_Tau22GM4, MSE_Tau22GM6)

print('#####')

print('Correlation coefficient between Sigma1 and GM2, GM4, GM6: ', corr_Sigma1GM2, corr_Sigma1GM4, corr_Sigma1GM6)
print('Mean squared error between Sigma1 and GM2, GM4, GM6: ', MSE_Sigma1GM2, MSE_Sigma1GM4, MSE_Sigma1GM6)
print('Correlation coefficient between Sigma2 and GM2, GM4, GM6: ', corr_Sigma2GM2, corr_Sigma2GM4, corr_Sigma2GM6)
print('Mean squared error between Sigma2 and GM2, GM4, GM6: ', MSE_Sigma2GM2, MSE_Sigma2GM4, MSE_Sigma2GM6)

print('#####')

print('Correlation coefficient between PiOmega and GM2, GM4, GM6: ', corr_PiOmegaGM2, corr_PiOmegaGM4, corr_PiOmegaGM6)
print('Mean squared error between PiOmega and GM2, GM4, GM6: ', MSE_PiOmegaGM2, MSE_PiOmegaGM4, MSE_PiOmegaGM6)

Calculate the correlations between the FDNS and GM2/4/6 SGS terms
Correlation coefficient between Tau11 and GM2, GM4, GM6:  0.9963032965656216 0.9988428994910861 0.9989300944502673
Mean squared error between Tau11 and GM2, GM4, GM6:  7.30805281789507e-07 2.0045164354320626e-07 1.775973272724732e-07
Correlation coefficient between Tau12 and GM2, GM4, GM6:  0.9955392451954392 0.9988654746701545 0.9990296322264454
Mean squared error between Tau12 and GM2, GM4, GM6:  3.1499084299426006e-07 7.94649112637057e-08 6.650210621448402e-08
Correlation coefficient between Tau22 and GM2, GM4, GM6:  0.995715523718232 0.9984291578041758 0.9985553040390178
Mean squared error between Tau22 and GM2, GM4, GM6:  1.4779709386042431e-06 4.668071744082806e-07 4.121949011423785e-07
#####
Correlation coefficient between Sigma1 and GM2, GM4, GM6:  0.9611283096111818 0.9717792049931708 0.9717184515466952
Mean squared error between Sigma1 and GM2, GM4, GM6:  0.00040510145163890243 0.00027359277137401026 0.00027471